<a href="https://colab.research.google.com/github/mimiachiu/Deep-Learning-with-Python-Book/blob/master/Ch08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
#下載和解析初始文字文件
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [ ]:
#向量化字元序列
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print(x.shape)
print(y.shape)

Number of sequences: 200278
Unique characters: 57
Vectorization...
(200278, 60, 57)
(200278, 57)


In [ ]:
#建立用於預測下一字元的模型
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
#建立從給定模型預測結果中取樣下一個字元的函式
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature #重新加權計算(熵)新的機率分布
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
#循環的文字生成
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y ,batch_size=128, epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    initial_text = text[start_index: start_index + maxlen]
    print('--- 隨機初始文字： "' + initial_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(initial_text)

        generated_text = initial_text
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
        sys.stdout.write('\n')

epoch 1
Epoch 1/1
200278/200278 [==============================] - 154s 769us/step - loss: 1.9719
--- 隨機初始文字： " of jealousy--ah, they know only too well that it will
flee "
------ temperature: 0.2
 of jealousy--ah, they know only too well that it will
flee and stander and the stands the precapses and the read and the presenty and man and the man the standers and the presential and will the extented the stand of the presentions and the great the carality of the present and present and the presented the stander and the man and stander the presented and the present of the stander the stands and a man and decistions and morality, and the extented and co
------ temperature: 0.5
 of jealousy--ah, they know only too well that it will
flee hearth the greates and must callies he proveser and stander of the same the seemed to the badfounderstands thinks to stind to the oblease the promants of the hame in the ration to the never such the present of southers of has a "is many the self to philosoph

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


m
unsign with at larkin, one must be gaas spey every che forehamenifice husiadil fru, the comprehen alift--andheir it usless immonstrou must it needing,--voicusly develytnessing themselves to the deem will
flacn.ges, is withoutged of closely certaint, whicher that sool are a developings for no feptsist and jesuy
werfi
epoch 15
Epoch 1/1
200278/200278 [==============================] - 166s 828us/step - loss: 1.3411
--- 隨機初始文字： "the heart desires,"
and in any case "beyond good and evil": "
------ temperature: 0.2
the heart desires,"
and in any case "beyond good and evil": the soul and states and the states and consequently and the sense of the same the words of the self-suffer to the same the sense of the strength of the sense of the self-respective to the same that the strength of the strength of the sense of the states and the self-suffering of the strength of the sublime the most precisely and the soul of the self-says of the same the same present to any precise
------ temperature: 0